In [1]:
from Reader import TimeMLReader
from datasets import DatasetDict

d:\GeoTKG\GeoTKG\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
S0 = TimeMLReader('./rawdata/Tempeval3/Training/TE3-Silver-data-0')
#intended_path = './cleandata/Tempeval3/silver-1.json'
train_dataset, label_list, label2id, id2label = S0.read(method="timex3_bio_tagger", return_data=True)

Processing file 1/400
Processing file 2/400
Processing file 3/400
Processing file 4/400
Processing file 5/400
Processing file 6/400
Processing file 7/400
Processing file 8/400
Processing file 9/400
Processing file 10/400
Processing file 11/400
Processing file 12/400
Processing file 13/400
Processing file 14/400
Processing file 15/400
Processing file 16/400
Processing file 17/400
Processing file 18/400
Processing file 19/400
Processing file 20/400
Processing file 21/400
Processing file 22/400
Processing file 23/400
Processing file 24/400
Processing file 25/400
Processing file 26/400
Processing file 27/400
Processing file 28/400
Processing file 29/400
Processing file 30/400
Processing file 31/400
Processing file 32/400
Processing file 33/400
Processing file 34/400
Processing file 35/400
Processing file 36/400
Processing file 37/400
Processing file 38/400
Processing file 39/400
Processing file 40/400
Processing file 41/400
Processing file 42/400
Processing file 43/400
Processing file 44/4

In [3]:
PL = TimeMLReader('./rawdata/Tempeval3/Evaluation/te3-platinum')
#intended_path = './cleandata/Tempeval3/platinum.json'
eval_dataset, label_list, label2id, id2label = PL.read(method="timex3_bio_tagger", return_data=True)

Processing file 1/20
Processing file 2/20
Processing file 3/20
Processing file 4/20
Processing file 5/20
Processing file 6/20
Processing file 7/20
Processing file 8/20
Processing file 9/20
Processing file 10/20
Processing file 11/20
Processing file 12/20
Processing file 13/20
Processing file 14/20
Processing file 15/20
Processing file 16/20
Processing file 17/20
Processing file 18/20
Processing file 19/20
Processing file 20/20


In [6]:
datasets = DatasetDict({
        "train": train_dataset,
        "eval": eval_dataset
    })

In [7]:
from Model import Model
import torch
from transformers import RobertaForTokenClassification, RobertaTokenizerFast, TrainingArguments

print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("Switching to GPU...")
    device = torch.device('cuda')
print("Current device index:", torch.cuda.current_device())
print("Device name:", torch.cuda.get_device_name(torch.cuda.current_device()))

CUDA available: True
Switching to GPU...
Current device index: 0
Device name: NVIDIA GeForce RTX 4070 Ti


In [13]:
# Load tokenizer and model
model_name = 'roberta-large'
tokenizer = RobertaTokenizerFast.from_pretrained(model_name, add_prefix_space=True)
model = RobertaForTokenClassification.from_pretrained(model_name, num_labels=len(label_list), label2id=label2id, id2label=id2label)

Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
training_args = TrainingArguments(
    output_dir="./results/Timex3-NER",
    logging_dir="./logs",
    eval_strategy="steps",
    save_strategy="steps",
    logging_steps=100,
    num_train_epochs=1,
    save_total_limit=1,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    learning_rate=5e-5,
    load_best_model_at_end=True,
    metric_for_best_model="f1"
)

model_instance = Model(
    device=device,
    model_name=model_name,
    tokenizer=tokenizer,
    model=model,
    label_list=label_list
)

model_instance.set_training_args(training_args)
tokenized_datasets = model_instance.tokenize_datasets(datasets)

Tokenizing datasets...


Map: 100%|██████████| 223/223 [00:00<00:00, 2764.46 examples/s]


In [17]:
model_instance.train(tokenized_datasets)


Starting training...


Step,Training Loss,Validation Loss,Precision,Recall,F1
100,0.046000,0.052321,0.757353,0.746377,0.751825


KeyboardInterrupt: 